# Evaluating the Utility of a Tesla Model S versus the Honda Odyssey

## Introduction
An individual decision-maker (agent) is given a set of $X_{1},\dotsc,X_{n}$ objects 
where each object has $m$ possible features $X_{i}=\left\{x_{1,i},\dotsc,x_{m,i}\right\}$.
A utility function ranks the agent's preference for combinations of features for each object $i$:

\begin{equation}
U_{i}(x_{1,i},x_{2,i},\dots,x_{m,i}) = u_{i}
\end{equation}

where $u_{i}$ is a real number called the `utility` for object $i$. Utility has units of `utils`. The utility function $U:X\rightarrow\mathbb{R}$ is unique only up to an order-preserving transformation. Utility functions are `ordinal`, i.e., they rank-order bundles but do not measure differences between bundles.

### Learning objectives
Fill me in

## Setup

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-5760-Examples-F23`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Manifest.toml`


In [2]:
function UCD(x)
    
    # load data -
    dataset = HondaTeslaDataSet();
    α = dataset[:,:exponent];
    
    number_of_features = length(x);
    solution = 1.0;

    # build the solution -
    for i ∈ 1:number_of_features
        solution *= x[i]^α[i];
    end

    # return - 
    return solution;
end

UCD (generic function with 1 method)

In [34]:
function MUCD(model::VLCobbDouglasUtilityFunction, features::Array{Float64,1})::Array{Float64,1}
    
    # initialize -
    number_of_features = length(features);
    values = zeros(number_of_features);
    α = model.α;
    
    for i ∈ 1:number_of_features
        tmp = prod(features[1:end .!=i].^(α[1:end .!=i]));
        values[i] = tmp*(α[i]*features[i]^(α[i]-1))
    end
    
    # return -
    return values
end

MUCD (generic function with 1 method)

## Data

In [21]:
dataset = HondaTeslaDataSet()
dataset[4,:Tesla] = 2.0;

dataset

Row,feature,exponent,Tesla,Honda
,String15,Float64,Float64,Float64
1,sustainability,0.2,5.0,3.0
2,affordability,0.1,2.0,4.0
3,styling,0.05,5.0,2.0
4,usefulness,0.3,2.0,5.0
5,costownership,0.1,4.0,2.0
6,performance,0.05,5.0,1.0
7,safety,0.2,5.0,5.0


In [4]:
@assert(sum(dataset[:,:exponent]) == 1.0)

### Build the Cobb-Douglas model
The `Cobb-Douglas` utility function is the product of the $m$ feature variables.  Thus, 
it models situations where we consider features simultaneously. Each feature variable is raised to a non-negative exponent:

\begin{equation}
U(x_{1},\dots,x_{m}) = \prod_{i\in{1\dots{m}}}{x_{i}^{\alpha_{i}}}
\end{equation}

In our realization of the `Cobb-Douglas` utility, the exponents must sum to unity $\sum_{i\in{1\dots{m}}}\alpha_{i} = 1$, $x_{i}\geq{0}$,
and $\alpha_{i}\geq{0}$. 

In [5]:
model = build(VLCobbDouglasUtilityFunction, (
        α = dataset[:,:exponent],)
);

In [22]:
utility_tesla = model(dataset[:,:Tesla]);
utility_honda = model(dataset[:,:Honda]);
println("The utility of Tesla = $(utility_tesla) while the utility of the Honda = $(utility_honda)")

The utility of Tesla = 3.389245277339831 while the utility of the Honda = 3.550338415119845


In [23]:
Δ = ((max(utility_tesla,utility_honda) - min(utility_tesla,utility_honda))/min(utility_tesla,utility_honda))*100

4.753068149333053

In [24]:
Ū_tesla = ForwardDiff.gradient(UCD, dataset[:,:Tesla]);
Ū_honda = ForwardDiff.gradient(UCD, dataset[:,:Honda]);

mu_table_df = DataFrame();
for i ∈ eachindex(Ū_tesla)
    
    row_tuple = (
        feature = dataset[i,:feature],
        S_Tesla = dataset[i,:Tesla],
        MU_Tesla = Ū_tesla[i],
        S_Honda = dataset[i,:Honda],
        MU_Honda = Ū_honda[i]
    );

    push!(mu_table_df, row_tuple);
end
mu_table_df

Row,feature,S_Tesla,MU_Tesla,S_Honda,MU_Honda
,String15,Float64,Float64,Float64,Float64
1,sustainability,5.0,0.13557,3.0,0.236689
2,affordability,2.0,0.169462,4.0,0.0887585
3,styling,5.0,0.0338925,2.0,0.0887585
4,usefulness,2.0,0.508387,5.0,0.21302
5,costownership,4.0,0.0847311,2.0,0.177517
6,performance,5.0,0.0338925,1.0,0.177517
7,safety,5.0,0.13557,5.0,0.142014


### Check: How good is the numerical estimate of the marginal utlity?

In [37]:
analytical_MU = MUCD(model,dataset[:,:Tesla]);
numerical_versus_analytical_df = DataFrame();
for i ∈ eachindex(Ū_tesla)
    
    row_tuple = (
        feature = dataset[i,:feature],
        S_Tesla = dataset[i,:Tesla],
        MU_N_Tesla = Ū_tesla[i],
        MU_A_Tesla = analytical_MU[i],
    );

    push!(numerical_versus_analytical_df, row_tuple);
end
numerical_versus_analytical_df

Row,feature,S_Tesla,MU_N_Tesla,MU_A_Tesla
,String15,Float64,Float64,Float64
1,sustainability,5.0,0.13557,0.13557
2,affordability,2.0,0.169462,0.169462
3,styling,5.0,0.0338925,0.0338925
4,usefulness,2.0,0.508387,0.508387
5,costownership,4.0,0.0847311,0.0847311
6,performance,5.0,0.0338925,0.0338925
7,safety,5.0,0.13557,0.13557


### Predicting Utility from feature changes
An individual decision-maker (agent) is given a set of $X_{1},\dotsc, X_{n}$ objects 
where each object has $m$ possible features $X_{i}=\left\{x_{1,i} \dotsc ,x_{m,i}\right\}$.
The _local change_ in the utility $U(\dots)$ near a point $x^{\star}\in{X}$ can be computed 
using the _total differential_ of the utility function $U(\dots)$:

\begin{equation}
dU = \sum_{i\in{1\dots{m}}}\bar{U}_{x_{i}}\cdot{dx_{i}}
\end{equation}

where $dU\approx\left(U - U^{\star}\right)$ denotes the local change in utility, 
$\bar{U}_{x_{i}}$ denotes the marginal utility of $x_{i}$ evaluated at the point $x^{\star}\in{X}$, 
and $dx_{i}\approx(x_{i}-x^{\star}_{i})$ is the change in the feature $x_{i}$. 

#### Application
In this example, the `usefulness` feature has the largest marginal utility for Tesla. What would happen to the Model S utility if Tesla were to increase the `usefulness` feature by `+1`? If all else was held constant (`omnibus paribus`), then the change of `dx` in a single feature `i` gives the new utility:

$$
\begin{equation}
U = U^{\star} + \bar{U}_{i}\cdot{dx_{i}}
\end{equation}
$$

In [31]:
dx = zeros(length(Ū_tesla));
dx[5] = 0.5;
U = utility_tesla + sum(Ū_tesla.*dx)

3.431610843306579